In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import os
import plotly.express as px
import plotly
import seaborn as sns
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from bs4 import BeautifulSoup
import spacy
import unidecode 
from keras_visualizer import visualizer
import contractions
from nltk.stem import WordNetLemmatizer 
import re
from tqdm import tqdm
nltk.download('wordnet')
POSTS = "posts"
TYPE = "type"

[nltk_data] Downloading package wordnet to /home/ivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
'''
This will load the csv
'''

class CsvToDf:
    '''
    This class will simply turn the given data to a dataframe
    '''
    def __init__(self,filename,batchSize=None,cols=None,preProc=False,postCol=False,toReplace=None):
        #batchSize is the size of data to be read incrementally. This is for data that is to big to fit
        #into memory
        self._toReplace = toReplace
        self._preProc = preProc
        self._postCol = postCol
        self._cols = cols
        self._header = None
        self._filename = filename
        self._curIndex = 0     #this will be the current index that we are in the csv
        self._isRead = False
        self._df = None
        self._storeHeader()
        self._batchSize = batchSize
    def preprocessing_v1(self,text):
        #remove html information
        if not(isinstance(text,str)):
            return text
        soup = BeautifulSoup(text, "html.parser")
        processed = soup.get_text(separator=" ")

        #remove http// 
        processed = re.sub(r"http\S+", "", processed)

        #remove ||| seperate
        processed = re.sub(r'\|\|\|', r' ', processed)

        #lower case
        processed = processed.lower()

        #expand shortened words, e.g. don't to do not
        processed = contractions.fix(processed)

        #remove accented char
        processed = unidecode.unidecode(processed)

        #remove white space
        #processed = processed.strip()
        #processed = " ".join(processed.split())

        # Lemmatizing 
        lemmatizer = WordNetLemmatizer() 
        processed=lemmatizer.lemmatize(processed)
        return processed
    def _storeHeader(self):
        with open(self._filename) as csvFile:
            f = csv.reader(csvFile)
            self._header = next(f)
    def getWholeCsv(self):
        if not(self._isRead):
            if self._cols != None:
                self._df = pd.read_csv(self._filename,usecols=self._cols)
            else:
                self._df = pd.read_csv(self._filename)
            self._isRead = True
        return self._df
    def getHeader(self):
        return self._header
    def _checkIfRead(self):
        if not(self._isRead):
            if self._cols != None:
                self._df = pd.read_csv(self._filename,iterator=True,chunksize=self._batchSize,usecols=self._cols)
            else:
                self._df = pd.read_csv(self._filename,iterator=True,chunksize=self._batchSize)
            self._isRead = True
            return False
        return True
    def removeWord(self,text):
        if not(isinstance(text,str)):
            return text
        words = text.split()
        cleanWords = [i for i in words if i not in self._toReplace]
        return " ".join(cleanWords)
    def getNextBatchCsv(self):
        self._checkIfRead()
        out = next(self._df,None)
        if self._preProc and isinstance(out,pd.DataFrame):
            out[self._postCol] = out[self._postCol].apply(self.preprocessing_v1)
        if self._toReplace != None and isinstance(out,pd.DataFrame):
            out[self._postCol] = out[self._postCol].apply(self.removeWord)
        return out

In [5]:
#================ counting the smallest number of data
TEST = "test"
TRAIN = "train"
class Combiner:
    '''
    - Given multiple CsvToDf that correspond to a dataset combine them to a single dataframe
    - return this dataframe
    - need to return a dataframe that only has type and post as its columns
    '''
    def __init__(self,dataList,columnList):
        '''
        dataList is the CsvToDf that contains all the data and columnList is a list that contains the necessary
        column names for a corresponding entry in dataList.
        '''
        assert len(dataList) == len(columnList),"incorrect sizes for data"
        self._dataList = dataList
        self._data = [None for i in range(len(dataList))]
        self._necessaryCol = columnList
        self._typeCol = "type"
        self._postCol = "posts"
        self._incrementData()
    def getNextBatch(self):
        '''
        return a dataframe that contains all the aggregated data
        '''
        outData = pd.DataFrame(columns=[self._typeCol,self._postCol])
        for data,colList in zip(self._data,self._necessaryCol):
            if isinstance(data,pd.DataFrame):
                renamedData = data[[colList[0],colList[1]]]
                renamedData.columns = [self._typeCol,self._postCol]
                
                outData = outData.append(renamedData,ignore_index=True)
        self._incrementData()
        if (len(outData.index)) == 0:
            return None
        else:
            return outData
    def _incrementData(self):
        for idx,i in enumerate(self._dataList):
            self._data[idx] = i.getNextBatchCsv()
class Balancer:
    '''
    - Balance the count
    - Decide what the training and test dat will be
    - Needs to output three data frames the train the test and the remainder
    - make the remainder the training set
    '''
    def __init__(self,combiner,trainFreq,testFreq):
        #personSize is minimum size of the number of people in a single personality group
        self._combiner = combiner
        self._typeCol = "type"
        self._postCol = "posts"
        self._personality_count = {"ENTJ" : {TRAIN:0,TEST:0}, "INTJ" : {TRAIN:0,TEST:0}, "ENTP" : {TRAIN:0,TEST:0}, "INTP" : {TRAIN:0,TEST:0}, "INFJ" : {TRAIN:0,TEST:0}, "INFP" : {TRAIN:0,TEST:0}, "ENFJ" : {TRAIN:0,TEST:0} , 
                    "ENFP" : {TRAIN:0,TEST:0}, "ESTP" : {TRAIN:0,TEST:0}, "ESTJ" : {TRAIN:0,TEST:0}, "ISTP" : {TRAIN:0,TEST:0}, "ISTJ" : {TRAIN:0,TEST:0}, "ISFJ" : {TRAIN:0,TEST:0}, "ISFP" : {TRAIN:0,TEST:0}, 
                    "ESFJ" : {TRAIN:0,TEST:0}, "ESFP" : {TRAIN:0,TEST:0}}
        self._trainFreq = trainFreq
        self._testFreq = testFreq
        self._training = []
        self._testing = []
    def createDataSets(self):
        self.reset()
        while not(self._trainIsUniform()) or not(self._testIsUniform()):
            #the three conditionals above will check if test and train dataset have uniform data 
            batch = self._combiner.getNextBatch()
            if not(isinstance(batch,pd.DataFrame)):
                break
            for idx,row in batch.iterrows():
                if isinstance(row[self._typeCol],str):
                    personality = row[self._typeCol].upper()
                    if personality in self._personality_count:
                        if self._personality_count[personality][TRAIN] < self._trainFreq:
                            self._training.append({self._typeCol:personality,self._postCol:row[self._postCol]})
                            self._personality_count[personality][TRAIN] += 1
                        elif self._personality_count[personality][TEST] < self._testFreq:
                            self._testing.append({self._typeCol:personality,self._postCol:row[self._postCol]})
                            self._personality_count[personality][TEST] += 1
        return True
    def reset(self):
        self._training = []
        self._testing = []
        self._personality_count = {"ENTJ" : {TRAIN:0,TEST:0}, "INTJ" : {TRAIN:0,TEST:0}, "ENTP" : {TRAIN:0,TEST:0}, "INTP" : {TRAIN:0,TEST:0}, "INFJ" : {TRAIN:0,TEST:0}, "INFP" : {TRAIN:0,TEST:0}, "ENFJ" : {TRAIN:0,TEST:0} , 
                    "ENFP" : {TRAIN:0,TEST:0}, "ESTP" : {TRAIN:0,TEST:0}, "ESTJ" : {TRAIN:0,TEST:0}, "ISTP" : {TRAIN:0,TEST:0}, "ISTJ" : {TRAIN:0,TEST:0}, "ISFJ" : {TRAIN:0,TEST:0}, "ISFP" : {TRAIN:0,TEST:0}, 
                    "ESFJ" : {TRAIN:0,TEST:0}, "ESFP" : {TRAIN:0,TEST:0}}
    def getTrainSet(self):
        return pd.DataFrame(self._training)
    def getTestSet(self):
        return pd.DataFrame(self._testing)
    def _trainIsUniform(self):
        #checks if personality count has equal distribution
        for key in self._personality_count:
            if self._personality_count[key][TRAIN] < self._trainFreq:
                return False
        return True
    def _testIsUniform(self):
        #checks if personality count has equal distribution
        for key in self._personality_count:
            if self._personality_count[key][TEST] < self._testFreq:
                return False
        return True
#======================================================

In [6]:
def getPersonalityDict():
    personality_dict = {"ENTJ" : 0, "INTJ" : 0, "ENTP" : 0, "INTP" : 0, "INFJ" : 0, "INFP" : 0, "ENFJ" : 0, 
                    "ENFP" : 0, "ESTP" : 0, "ESTJ" : 0, "ISTP" : 0, "ISTJ" : 0, "ISFJ" : 0, "ISFP" : 0, 
                    "ESFJ" : 0, "ESFP" : 0}
    for idx,keys in enumerate(personality_dict):
        oneVec = np.zeros((16,))
        oneVec[idx] = 1
        personality_dict[keys] = oneVec
    return personality_dict

In [7]:
def counterDf(df):
    personality_dict = {"ENTJ" : 0, "INTJ" : 0, "ENTP" : 0, "INTP" : 0, "INFJ" : 0, "INFP" : 0, "ENFJ" : 0, 
                    "ENFP" : 0, "ESTP" : 0, "ESTJ" : 0, "ISTP" : 0, "ISTJ" : 0, "ISFJ" : 0, "ISFP" : 0, 
                    "ESFJ" : 0, "ESFP" : 0}

    for idx,row in df.iterrows():
        if isinstance(row["type"],str):
            personality = row["type"].upper()
            if personality in personality_dict:
                personality_dict[personality] += 1
    return personality_dict

In [8]:
TYPE = "type"
def convertLabels(labelDf):
    '''
    this will turn the string labels to floats
    '''
    personality_dict = getPersonalityDict()
    type_labels = []
    # Go through the array and turn the personality type into its corresponding number
    for idx,personality in enumerate(labelDf):
        if isinstance(personality,str):
            type_labels.append(personality_dict[personality.upper()])
    return np.array(type_labels)

In [9]:
TYPE = "type"
def get4Dim(df):
    personality_dict = {"ENTJ" : 0, "INTJ" : 0, "ENTP" : 0, "INTP" : 0, "INFJ" : 0, "INFP" : 0, "ENFJ" : 0, 
                    "ENFP" : 0, "ESTP" : 0, "ESTJ" : 0, "ISTP" : 0, "ISTJ" : 0, "ISFJ" : 0, "ISFP" : 0, 
                    "ESFJ" : 0, "ESFP" : 0}
    out = [[0 for i in range(len(df.index))],[0 for i in range(len(df.index))],[0 for i in range(len(df.index))],[0 for i in range(len(df.index))]]
    for idx,row in enumerate(df):
        personality = row
        if isinstance(personality,str) and personality in personality_dict:
            personality = personality.upper()
            if personality[0] == "E":
                out[0][idx] = 1
            if personality[1] == "S":
                out[1][idx] = 1
            if personality[2] == "T":
                out[2][idx] = 1
            if personality[3] == "J":
                out[3][idx] = 1
    return out

In [3]:
embedding_vector = {}
f = open('../glove.twitter.27B/glove.twitter.27B.25d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef


1193515it [00:09, 128866.22it/s]


In [5]:
for i in embedding_vector:
    print(i)

<user>
.
:
rt
,
<repeat>
<hashtag>
<number>
<url>
!
i
a
"
the
?
you
to
(
<allcaps>
<elong>
)
me
de
<smile>
！
que
and
。
-
my
no
、
is
it
…
in
n
for
/
of
la
's
*
do
n't
that
on
y
'
e
o
u
en
this
el
so
be
'm
with
just
>
your
^
like
have
te
at
？
love
se
are
<
m
r
if
all
b
・
not
but
we
es
ya
&
follow
up
what
get
lol
un
♥
lo
when
was
“
”
one
por
si
out
_
mi
can
<sadface>
من
♡
´
he
con
they
now
go
،
para
los
know
haha
good
tu
back
~
about
new
;
as
day
how
who
will
want
people
yo
eu
from
di
time
<heart>
s
aku
da
're
<lolface>
una
got
las
more
x
she
today
（
>>
k
by
or
في
･
too
le
é
|
[
）
]
see
why
yg
ca
como
her
—
q
need
an
na
笑
there
ω
happy
im
mas
je
life
really
make
yang
shit
think
t
❤
não
never
some
～
oh
★
did
would
del
`
d
please
via
much
fuck
al
dia
$
و
right
best
c
going
الله
pero
only
has
♪
'll
twitter
=
hahaha
its
nn
｀
¿
am
say
<neutralface>
them
here
لا
off
still
dan
+
night
w
ada
someone
even
then
☆
ni
come
com
always
man
've
been
his
itu
على
-_-
☺
over
um
ما
hate
girl
ai
had
pra
todo

mexico
confused
bawa
moon
review
company
pela
view
doctor
perasaan
li
jugar
gobierno
foda
cowok
pie
terrible
france
ну
goodbye
felicidad
agua
kurang
dele
score
saudade
cheese
problemas
bill
humor
tek
positive
energy
excuse
dancing
experience
º
السعودية
lyrics
entrar
regret
brothers
entiendo
tes
nda
award
lips
feo
相互フォロー
mı
festival
airport
completely
pobre
virgo
winner
directioner
jump
اي
married
doin
surveys
camera
sum
release
wen
malas
ahead
somewhere
pueblo
fool
□
equipo
toma
viu
oppa
idiota
amigas
la.
december
mente
bareng
tao
butuh
romney
ala
disini
mike
primero
noo
cierto
paid
hablando
aime
ape
:@
writing
класс
oye
usually
baju
trending
keluar
mag
nangis
entire
system
adoro
except
camino
canada
kawan
eve
windows
اكثر
ft.
famous
toca
brazil
books
acting
victoria
tattoo
deu
often
gaga
mobile
verte
wala
❌
maen
pisces
performance
kn
french
maria
marry
hat
law
jomblo
kasi
galaxy
deixa
kmu
musicfans
error
reality
action
valentines
tous
الجنة
stfu
cam
حين
hr
dolor
sigueme
الذين
duit
bas

viver
diciendo
⚽
value
rick
̶
loose
hitting
maunya
alert
namun
glasses
visita
temporada
petit
ninguna
washington
stone
feelin
artis
msg
quiser
thomas
jorge
gross
cinema
becomes
fuiste
birth
التزم
abg
banco
asks
awe
kpn
بالرتويت
quede
pergunta
cade
puntos
horny
spongebob
virgin
guns
ghost
gangnam
⭐
pasando
aaah
hee
aff
achei
israel
mii
pueda
mto
loser
kong
views
otak
algun
أنه
perdón
posta
türk
avoid
anti
jaa
hebat
util
ptdr
bj
البشر
liars
largo
pressure
ellas
الموت
hij
raining
dulce
hacerlo
genius
genre
worries
menangis
надо
minds
toronto
oscars
eta
cuál
mam
fitness
nena
estudiantes
lock
smells
bravo
constantly
entao
sao
̅
forte
cape
fancy
idgaf
dmn
지금
response
sexta
puro
mimin
mexican
tarea
risk
gabisa
gotten
dp
twitcam
bull
proper
تعرف
احبك
emma
՞
agak
crowd
llevar
ま
actor
tayo
houston
vei
gn
:]
grazie
library
respeto
smua
barbie
bunu
iku
k.
alles
근데
crisis
liburan
bum
было
according
content
valla
liar
чем
solat
bayar
prefiero
accounts
anne
اضغط_فولو
quickly
muere
carnaval
correr
war

stunning
instantly
عنها
replied
vestido
aamiin
خالد
し
ultra
ru
pgn
señores
cuts
ｗｗ
rachel
classe
vish
новости
broncos
financial
poxa
standard
nian
taun
agama
tambem
goreng
gabriel
rafael
pays
kehidupan
بوك
眠い
واحدة
⚫
highly
лучше
humble
santiago
among
muhammad
twitter自動つぶやき設定ができるサイト
samasama
piso
huevo
valley
真顔
complicated
secara
trey
fails
يوماً
parque
لماذا
improve
cries
bodo
nh
leather
kardashian
'cause
オートツイート
piscina
jadian
¥
cuti
mmm
ebay
rugby
nowadays
demas
tod
pelotudo
weave
caras
completo
hush
desire
nenhum
federal
定期的につぶやくbotやurlの宣伝にも使えます
hurricane
cadê
jugador
игры
だから
letras
varsa
bale
jaar
شكرا
podem
że
العالمين
يريد
niya
assistindo
kamus
الفيس
pé
musim
foreign
nine
aprendí
móvil
sweetest
ulangan
pretending
entérate
dip
pollo
direito
mrs
ngm
tahan
enemies
maría
berasa
chino
fuk
doet
addiction
saiu
models
libros
infancia
funciona
manage
leva
ahahahah
хорошо
segura
bones
thf
purchase
sente
profe
ipa
يعلم
stands
speaks
ova
»»
دائما
الأشياء
nigeria
cadena
parade
pops
takiped

yeay
aids
borrow
reforma
tommy
kidrauhl
ours
hahahahahahahaha
للمتواجدينn
cenar
aulas
tata
dieta
jornada
grad
uhuk
kazan
interesa
stripper
jua
pendiente
ヤフオク
tuits
marcelo
releases
seguime
ｉ
ditunggu
supply
كلهم
analysis
nabi
geceler
therapy
vont
mickey
maret
قدر
smurfs
puente
entertaining
الورد
sabi
jajajajajajajajajaja
slam
yankees
dejame
oso
duo
rvp
doit
❒
eski
الأمل
neighbors
detik
typing
escribo
curly
ajajaja
replies
pills
dentist
sincerely
brazos
acorda
gap
podcast
☂
汗
benefit
survey
bil
akuu
rockin
tamo
intelligence
فمن
kerjaan
blank
делать
veremos
cambian
بكره
shed
meskipun
schon
lepak
fx
januari
piro
sticks
selasa
recibir
marc
bolt
jajja
ただ
barbara
committed
terbaru
jogando
nurse
usd
r.
artik
الهم
白目
çocuk
mintak
телефон
spice
corriendo
burnt
spray
ajax
pesan
fart
decía
lalala
wi-fi
preocupes
roads
pace
الشيطان
ninguno
río
suma
相互
rivera
kulit
bothered
ｪ
gate
popped
nombres
hbu
twerking
sucesso
ski
suspended
impresionante
afim
geh
olmuş
uts
بشار
conditions
recognize
everton
pl

...

----------------
bife
dasom
diğeri
sicak
أسد
وانـا
allée
chers
g'morning
πω
كلثوم
했지만
actuacion
belia
despacho
dituduh
flacos
rebuilding
биболетова
bretagne
dirasa
eventhough
поговорить
devs
enne
gash
hummels
noting
ちゅー
adayn
enjoada
joc
kiddies
duniaku
hellokitty
بالجوال
inge
mentem
ดี
客
일요일
rebo
لغات
concepcion
jajajajjajajajaja
obscure
التنمية
فأغفر
訂正
brille
cartelera
dasi
mahe
notified
overuse
pedaços
spielen
선
bodied
perteneces
sencillez
쭉
okuyup
provato
berhati-hatilah
boleeh
meld
cónclave
diita
fijas
автомобилей
حيلة
自民
chocan
entretenida
krj
seguirlos
ก.พ
경수
blusas
lamest
rosales
年のツイートを分析して
이글
asistencias
añadir
kecilnya
pinjaman
snag
اللطيف
왜케
jupp
rolas
spettacolo
uuf
وعيني
nuno
sonntag
дороге
bastardo
بخدمتك
bruselas
denso
nikt
panto
timen
zebras
ｵ
endorsed
hunnid
nvamos
recorre
salitang
tengáis
ülker
すんません
bayrağı
hiraukan
homosexuals
arrancando
biten
capabilities
carbonero
ciiie
lke
meko
orait
padu
pretentious
미리보기
draxler
oficialista
числа
وزيادة
サンケイスポーツ
ayudara
feiten

lettres
overleden
ahahahahahahahah
bersenang
pulpo
telle
uchiha
beeb
fracture
fuéramos
mantenga
pangan
paradiso
yazılır
yokum
бабы
backlash
chodzi
maicih
pésima
uscita
أناقة
♘
お腹すいたー
노래가
열
odee
wua
الجليل
مماتك
وآمن
والملائكة
binding
curiosos
merendah
readyy
احرار
بالحزن
alegrado
bayou
farkettim
pesimista
لبست
cremas
developments
didoain
kwkwkwk
سار
كرستيانو
diagonal
icat
membayangkan
samy
bidik
faya
hahahaahaha
rimel
vriendinnetje
yelahh
твитов
условия
بدور
deity
gorrito
relevance
religioso
seeya
teyus
送料別
dersiniz
detuvo
avante
değilmiş
folds
lpdla
parejo
prestando
resfriada
çokta
рост
kaut
porcentaje
sobrancelha
uacho
تراهم
confere
mfc
sucesos
yoan
угодно
おかしいな
anny
ansa
encierro
guardate
harmoni
nce
تجتمع
hacee
ilusione
smashin
tribunales
directeur
heatwave
kuching
nحساب
pneumonia
terbuang
سقيا
فقراء
chime
أجرنا
تبع
حيلي
akit
antep
brisk
krupuk
menstrual
optimal
pasarán
timely
بوحي
وأبقى
guaira
mallory
revenu
skyrock
tinted
встать
الذاكره
そのうち
creerse
wkwrt
идем
بحيث
그렇구나
alesso
ga

oen
غدى
almohadas
istemiyoruz
jadakiss
magnífico
menemui
pulseira
عاجلاً
梅田
結構
보내고
badder
blige
censor
creampie
instagramers
jnji
rıza
viki
yoyoy
تصغر
يقتنع
bosver
franks
hoppin
jodidas
köprü
ventures
фотка
الدنيـا
السمك
handler
memperoleh
n'existe
razer
stessi
trött
ترم
boum
bwisit
gramado
hamboi
martinelli
pasaré
༩
すみませんでした
salla
subas
whata
افكار
العيوب
n'oublie
soñaste
swelling
zaa
نجح
hails
jles
kykny
pemprov
نومة
等
canalla
diyolar
musiquinha
perds
tovar
لهَا
dönüyor
estética
feby
quítate
الوصف
شنطة
مبادرة
▊
doya
inanılmaz
ngaps
pelet
tablette
بحس
ヤフー
güncel
muridnya
organik
vowels
حديثاً
عملنا
محامي
大丈夫ー
aspettare
charot
fearful
lue
montones
العبيد
علی
feta
goodnightt
impromptu
motions
nちょ
alejas
ekip
exotics
guai
j.j.
kesalahannya
ʔु
елена
حاس
aip
coomo
llegate
menaklukan
nnكيف
picos
uba
vendu
ʃƪ˘
выхода
прошли
داخلية
アホ
abertos
confesión
garantitakip
mythology
рай
الكعبه
مريح
相互フォローよろしくお願いします
attain
conseguiram
excused
genocida
leni
soirées
spices
už
останется
cubic
fern
impuls

prioridade
ʷʰʸn
سأظل
قبرها
baliknya
dorset
membebel
sofrem
إنتظار
gruden
kean
ñe
пишешь
有
ajjajajajaja
bunlari
buzzfeed
cilacap
deta
disposto
dwa
haaan
mengorbankan
nonostante
pintarme
rumh
الألف
الشباك
angulo
duchar
gmpang
o'reilly
subsidies
wayah
ответить
اموات
سجده
あや
当選者の方には責任をもってdmでご連絡致します
llevaré
ofrecemos
tarifi
tincho
ugliness
автобусе
يمنعك
これを
completas
escolheria
ezan
gloucester
maliki
membros
korkunç
nب
r.o.b.
raves
veq
bilsin
n.n.n.
obtén
reciba
牛乳
footlong
gedek
huskers
tatau
أمراض
الشئ
توتنهام
butik
poseidon
лежу
ceniza
سألوا
adoram
aportar
chilean
competição
semáforos
نحبك
ورحت
يجرحك
本名
hatur
ميزة
asadito
inconsciente
morti
sagala
w.w
şerif
المشكلات
بہا
カゲプロ好きな人rt
나눔
chameleon
d.f.
juggling
lavarse
mbakk
nmor
علك
مليارات
생일축하해
aguacate
forgetful
mildly
التميز
باشا
birliği
cortázar
fos
halini
myung
pacare
spelers
varnish
بنورك
تكرر
للايفون
ajuste
blaster
cuídala
nn彼氏
بصحة
방
ﾟдﾟщ
nns
relajada
seguii
لحملة
melis
tunel
yua
أراه
رفيقي
başkaları
calçada
maldigo
sketchers
stop

KeyboardInterrupt: 

In [22]:
def createDimModel():
    vocab_size = 10000
    max_length = 2016
    embedding_dim = 256
    return tf.keras.Sequential([ 
                            tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                            tf.keras.layers.GRU(64, return_sequences=True),
                            tf.keras.layers.SimpleRNN(64),
                            tf.keras.layers.Dense(1, activation='sigmoid'),])
def train4Dim(trainPost,trainLabels,num_epochs):
    models = [createDimModel(),createDimModel(),createDimModel(),createDimModel()]
    for idx,dims in enumerate(trainLabels):
        print(f"training dim {idx+1}")
        models[idx].compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics = ["accuracy"])
        models[idx].fit(trainPost, np.array(trainLabels[idx]), epochs = num_epochs, verbose = 1)
    return models
def test4Dim(models,testPost,testLabels):
    out = np.array([])
    for idx,model in enumerate(models):
        if len(out) == 0:
            out = np.array([np.squeeze(model.predict(testPost))])
        else:
            print(out)
            out = np.append(out,np.squeeze(model.predict(testPost)),axis=1)
    label = np.array([[]])
    for i in testPost:
        if len(out) == 0:
            label = np.array([label])
        else:
            label = np.append(label,i,axis=1)
    print(f"total accuracy for personality classification = {np.mean((np.sum(label-out,axis=1)) == 0)}")

In [11]:
def removeWord(se,text):
        if not(isinstance(text,str)):
            return text
        words = text.split()
        cleanWords = [i for i in words if i not in self._toReplace]

In [12]:
def dfFactory(fileList,columns,replaceWords,personTrain):
    '''
    This will create a dataframe without the specified words
    '''
    files = []
    for idx,i in enumerate(fileList):
        files.append(CsvToDf(i,batchSize=400,preProc=True,postCol=columns[idx][1],toReplace=replaceWords))
    combine = Combiner(files,columns)
    balancer = Balancer(combine,personTrain,0)
    balancer.createDataSets()
    return balancer

In [13]:
vocab_size = 10000
max_length = 2016
def tokenize(postsSet):
    tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>")
    tokenizer.fit_on_texts(postsSet)
    training_sequences = tokenizer.texts_to_sequences(postsSet)
    training_padded = pad_sequences(training_sequences, padding = 'post', maxlen = 2016)
    # training_sequences = np.array(training_sequences)
    training_padded = np.array(training_padded)
    return training_padded

In [14]:
def createTrain(fileList,columns,replaceWords,personTrain):
    out = []
    for i in replaceWords:
        balancer = dfFactory(fileList,columns,i,personTrain)
        out.append(balancer.getTrainSet())
    return out

In [14]:

def createModels(trainSets,toReplace):
    '''
    trainSets must be a list of dataframes with columns posts and type
    '''
    out = []
    for idx,i in enumerate(trainSets):
        print(f"words removed: {toReplace[idx]})
        out.append(train4Dim(tokenize(i[POSTS]),get4Dim(i[TYPE]),1))
    return out

SyntaxError: EOL while scanning string literal (<ipython-input-14-1f8764a73df4>, line 7)

In [ ]:
def testModels(models):
    '''
    models will be a list of list containing a model for each dimension and each model with some words taken out
    '''
    for i in models:
        test4Dim(i)

In [23]:
file1 = CsvToDf("../data/mbti_1.csv")

combine = Combiner([file1],[["type","posts"]])
balancer = Balancer(combine,39,30)
balancer.createDataSets()

True

In [128]:
file1 = CsvToDf("../data/mbti_1.csv")
training_size = 6675
df = file1.getWholeCsv()
training_labels = get4Dim(df[0:training_size]["type"])
testing_labels = get4Dim(df[training_size:]["type"])
testing_posts = df[training_size:]["posts"]
training_posts = df[0:training_size]["posts"]

In [24]:
training_labels = get4Dim(balancer.getTrainSet()["type"])
testing_labels = get4Dim(balancer.getTestSet()["type"])
testing_posts = balancer.getTestSet()["posts"]
training_posts = balancer.getTrainSet()["posts"]

In [25]:
# Only considering the top 10000 most common words
vocab_size = 10000
max_length = 2016
# We only want to fit the tokenizer on the training, not the testing
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(training_posts)

word_index = tokenizer.word_index

# Puts the padding (which are 0) at the end of the vectorized sentence.
# The longest post in our dataset is 2016, but we should truncate='post' earlier than 2016 words
training_sequences = tokenizer.texts_to_sequences(training_posts)
training_padded = pad_sequences(training_sequences, padding = 'post', maxlen = max_length)
# training_sequences = np.array(training_sequences)
training_padded = np.array(training_padded)


testing_sequences = tokenizer.texts_to_sequences(testing_posts)
testing_padded = pad_sequences(testing_sequences, padding = 'post', maxlen=max_length)
# testing_sequences = np.array(testing_sequences)
testing_padded = np.array(testing_padded)



In [27]:
#this is using a balanced subset of the dataset for training only
models = train4Dim(training_padded,training_labels,2)

training dim 1
Epoch 1/2
20/20 [==============================] - 42s 2s/step - loss: 0.7054 - accuracy: 0.4550
Epoch 2/2
20/20 [==============================] - 42s 2s/step - loss: 0.6976 - accuracy: 0.4945
training dim 2
Epoch 1/2
20/20 [==============================] - 42s 2s/step - loss: 0.7079 - accuracy: 0.4621
Epoch 2/2
20/20 [==============================] - 42s 2s/step - loss: 0.6999 - accuracy: 0.5170
training dim 3
Epoch 1/2
20/20 [==============================] - 42s 2s/step - loss: 0.7049 - accuracy: 0.4872
Epoch 2/2
20/20 [==============================] - 41s 2s/step - loss: 0.6901 - accuracy: 0.5570
training dim 4
Epoch 1/2
20/20 [==============================] - 37s 2s/step - loss: 0.7090 - accuracy: 0.4559
Epoch 2/2
20/20 [==============================] - 39s 2s/step - loss: 0.6991 - accuracy: 0.5148


In [18]:
#this is using a balanced subset of the dataset
models = train4Dim(training_padded,training_labels,5)

training dim 1
Epoch 1/5
15/15 [==============================] - 23s 1s/step - loss: 0.6982 - accuracy: 0.5305
Epoch 2/5
15/15 [==============================] - 21s 1s/step - loss: 0.6976 - accuracy: 0.5216
Epoch 3/5
15/15 [==============================] - 22s 1s/step - loss: 0.6933 - accuracy: 0.5131
Epoch 4/5
15/15 [==============================] - 21s 1s/step - loss: 0.6958 - accuracy: 0.5207
Epoch 5/5
15/15 [==============================] - 22s 1s/step - loss: 0.6947 - accuracy: 0.4890
training dim 2
Epoch 1/5
15/15 [==============================] - 23s 1s/step - loss: 0.7071 - accuracy: 0.4790
Epoch 2/5
15/15 [==============================] - 23s 2s/step - loss: 0.7138 - accuracy: 0.4999
Epoch 3/5
15/15 [==============================] - 21s 1s/step - loss: 0.6945 - accuracy: 0.5364
Epoch 4/5
15/15 [==============================] - 22s 1s/step - loss: 0.6986 - accuracy: 0.5044
Epoch 5/5
15/15 [==============================] - 22s 1s/step - loss: 0.6962 - accuracy: 0.4365


In [137]:
#this is using the entire dataset
models = train4Dim(training_padded,training_labels,5)

training dim 1
Epoch 1/5
209/209 [==============================] - 434s 2s/step - loss: 0.5537 - accuracy: 0.7514
Epoch 2/5
209/209 [==============================] - 435s 2s/step - loss: 0.5443 - accuracy: 0.7681
Epoch 3/5
209/209 [==============================] - 373s 2s/step - loss: 0.5382 - accuracy: 0.7720
Epoch 4/5
209/209 [==============================] - 421s 2s/step - loss: 0.5395 - accuracy: 0.7715
Epoch 5/5
209/209 [==============================] - 342s 2s/step - loss: 0.5435 - accuracy: 0.7676
training dim 2
Epoch 1/5
209/209 [==============================] - 434s 2s/step - loss: 0.4170 - accuracy: 0.8409
Epoch 2/5
209/209 [==============================] - 437s 2s/step - loss: 0.4077 - accuracy: 0.8599
Epoch 3/5
209/209 [==============================] - 436s 2s/step - loss: 0.3896 - accuracy: 0.8691
Epoch 4/5
209/209 [==============================] - 438s 2s/step - loss: 0.3951 - accuracy: 0.8658
Epoch 5/5
209/209 [==============================] - 429s 2s/step - lo

In [28]:

def getAccuracy(models,testing_labels):
    for idx,model in enumerate(models):
        test = testing_labels[idx]
        modelOut = np.round(models[idx].predict(testing_padded))
        print(f"accuracy for dim {idx+1} personality classification = {np.mean(abs(np.squeeze(modelOut)-np.squeeze(test)) == 0)}")
getAccuracy(models,testing_labels)

accuracy for dim 1 personality classification = 0.40298507462686567
accuracy for dim 2 personality classification = 0.44776119402985076
accuracy for dim 3 personality classification = 0.47761194029850745
accuracy for dim 4 personality classification = 0.4552238805970149


In [30]:
def getTotalAccuracy(models,testing_labels):
    total = None
    isEmpty = True
    for idx,model in enumerate(models):
        test = testing_labels[idx]
        modelOut = np.squeeze(np.round(models[idx].predict(testing_padded)))
        if isEmpty:
            total = np.array(modelOut)
            isEmpty = False
        else:
            total = np.column_stack((total,modelOut))
    labels = None
    isEmpty = True
    for idx,col in enumerate(testing_labels):
        if isEmpty:
            labels = np.array(col)
            isEmpty = False
        else:
            labels = np.column_stack((labels,col))
    print(f"total accuracy for personality classification = {np.mean(np.sum(abs(total-labels),axis=1) == 0)}")
getTotalAccuracy(models,testing_labels)

total accuracy for personality classification = 0.03233830845771144


In [31]:
getAccuracy(models,testing_labels)
getTotalAccuracy(models,testing_labels)

accuracy for dim 1 personality classification = 0.40298507462686567
accuracy for dim 2 personality classification = 0.44776119402985076
accuracy for dim 3 personality classification = 0.47761194029850745
accuracy for dim 4 personality classification = 0.4552238805970149
total accuracy for personality classification = 0.03233830845771144


In [9]:
#Second parameter is the output dimension. Therefore, when we are changing this to predict 4 dimensions of personality we should change it to 4
# ^^ actually i dont know if that is true
embedding_dim = 256
'''
Embedding layer will always have vocab_size*embedding_dim parameters. Since vocab_size is 10,000 the number of parameters on this layer will always be large
'''
model = tf.keras.Sequential([ 
                            tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                            tf.keras.layers.GRU(256, return_sequences=True),
                            tf.keras.layers.SimpleRNN(128),
                            tf.keras.layers.Dense(16, activation='softmax'),
])

model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer = 'sgd', metrics = ["accuracy"])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2016, 256)         2560000   
_________________________________________________________________
gru (GRU)                    (None, 2016, 256)         393984    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               49280     
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
Total params: 3,005,328
Trainable params: 3,005,328
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
history = model.fit(training_padded, training_labels, epochs = num_epochs, verbose = 1)

In [ ]:
history = model.fit(training_padded, training_labels, epochs = 1, verbose = 1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 128/1600 [=>............................] - ETA: 3:03 - loss: 2.8088 - acc: 0.0547

In [ ]:
res = np.argmax(model.predict(testing_padded),axis=1)
label = np.argmax(testing_labels,axis=1)
print(f"accuracy = {np.mean((label-res) == 0)}")

In [35]:
vizModel = createDimModel()
visualizer(vizModel,format="png",view=True)

ValueError: Keras Visualizer: Layer not supported for visualizing